In [70]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd '/content/gdrive/MyDrive/Lumbar-spline-from-ishmeet'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Lumbar-spline-from-ishmeet


In [ ]:
%pip install torch torchvision

In [10]:
from PIL import Image
import torch
from torchvision import transforms
import torchvision
from torch import nn

In [11]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
device = "cuda" if torch.cuda.is_available() else "cpu"
class_names = ['processed_lsd_jpgs',
 'processed_osf_jpgs',
 'processed_spider_jpgs',
 'processed_tseg_jpgs']

In [12]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
for param in model.features.parameters():
    param.requires_grad = False

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names),
              bias=True).to(device))

model.load_state_dict(torch.load('model_final_weights.pth'))

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\usar/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100.0%
C:\Users\usar\AppData\Local\Temp\ipykernel_9620\3134101881.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't hav

<All keys matched successfully>

In [15]:
def predict_image_class(image_path, model, class_names, device='cpu'):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)

    input_batch = input_batch.to(device)
    model.eval()

    with torch.no_grad():
        output = model(input_batch)
        _, predicted_idx = torch.max(output, 1)

    predicted_class = class_names[predicted_idx.item()]
    return predicted_class

In [17]:
predict_image_class('data/processed_spider_jpgs/1_t2.jpg', model, class_names, device='cpu')

'processed_spider_jpgs'

In [ ]:
# %pip install flask-ngrok

In [ ]:
# from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
  if 'image' not in request.files:
        return "No image is uploaded !"
  file = request.files["image"]
  file.save("static/input.jpg")
  return render_template("output.html", result=predict_image_class('static/input.jpg', model, class_names, device='cpu'))

# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(5000)"))

app.run()
# run_with_ngrok(app)

In [21]:
from flask import Flask, request, render_template
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
  if 'image' not in request.files:
        return "No image is uploaded !"
  file = request.files["image"]
  file.save("static/input.jpg");
  return render_template("output.html", result=predict_image_class('static/input.jpg', model, class_names, device='cpu'))

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Oct/2024 20:30:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2024 20:30:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/Oct/2024 20:31:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [10/Oct/2024 20:31:04] "GET /static/input.jpg HTTP/1.1" 200 -
